# SerpAPI Tool Helper

This notebook provides a quick rundown of how we construct an async SerpAPI tool. Let's start by query SerpAPI synchronously via their Python SDK:

In [ ]:
from serpapi import GoogleSearch
from getpass import getpass

SERPAPI_API_KEY = getpass("Enter your SerpAPI API key: ")

params = {
    "api_key": SERPAPI_API_KEY,
    "engine": "google",
    "q": "latest news in the world",
}

search = GoogleSearch(params)
results = search.get_dict() #这一行表明是同步的，内部会调用类似requests.get()

print(results)

{'search_metadata': {'id': '68762e52895d90ddd276d1f5', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/6e396d47fea95486/68762e52895d90ddd276d1f5.json', 'pixel_position_endpoint': 'https://serpapi.com/searches/6e396d47fea95486/68762e52895d90ddd276d1f5.json_with_pixel_position', 'created_at': '2025-07-15 10:32:50 UTC', 'processed_at': '2025-07-15 10:32:50 UTC', 'google_url': 'https://www.google.com/search?q=latest+news+in+the+world&oq=latest+news+in+the+world&sourceid=chrome&ie=UTF-8', 'raw_html_file': 'https://serpapi.com/searches/6e396d47fea95486/68762e52895d90ddd276d1f5.html', 'total_time_taken': 1.28}, 'search_parameters': {'engine': 'google', 'q': 'latest news in the world', 'google_domain': 'google.com', 'device': 'desktop'}, 'search_information': {'query_displayed': 'latest news in the world', 'total_results': 1540000000, 'time_taken_displayed': 0.31, 'organic_results_state': 'Results for exact spelling'}, 'inline_videos': [{'position': 1, 'title': 'ABC World N

Our results are provided in the `"organic_results"` key:

In [2]:
results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjcpPni1L6OAxW2LtAFHTrRJg4QFnoECFMQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/68762e52895d90ddd276d1f5/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'Get all the latest news, live updates and content about the World from across the BBC.',
  'snippet_highlighted_words': ['latest news', 'World'],
  'sitelinks': {'inline': [{'title': 'BBC World', 'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East', 'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Europe', 'link': 'https://www.bbc.com/news/world/europe'},
    {'title': 'Africa', 'link': 'https://www.bbc.com/news/world/africa'}]},
  'source': 'BBC'},
 {'p

We reformat this to extract only the most relevant information, such as the the title, source, link, and snippet. Let's use pydantic `BaseModel` to define this structure.

In [3]:
from pydantic import BaseModel

class Article(BaseModel):
    title: str
    source: str
    link: str
    snippet: str

    @classmethod
    def from_serpapi_result(cls, result: dict) -> "Article":
        return cls(
            title=result["title"],
            source=result["source"],
            link=result["link"],
            snippet=result["snippet"],
        )

In [4]:
articles = [Article.from_serpapi_result(result) for result in results["organic_results"]]

In [5]:
articles

[Article(title='World | Latest News & Updates', source='BBC', link='https://www.bbc.com/news/world', snippet='Get all the latest news, live updates and content about the World from across the BBC.'),
 Article(title='World news - breaking news, video, headlines and opinion', source='CNN', link='https://www.cnn.com/world', snippet='View CNN world news today for international news and videos from Europe, Asia, Africa, the Middle East and the Americas.'),
 Article(title='World news', source='CBS News', link='https://www.cbsnews.com/world/', snippet='World · Nvidia says Trump admin. · Trump administration imposes 17% tariff on fresh Mexican tomatoes · Aristocrat and her partner found guilty over baby ...'),
 Article(title='Breaking News, World News and Video from Al Jazeera', source='Al Jazeera', link='https://www.aljazeera.com/', snippet="Al Jazeera · Cricket: England win dramatic Lord's Test against India by 22 runs · Chelsea trounce PSG to lift FIFA Club World Cup · Revenge is ..."),
 Ar

## Using Async

All of this works, but it unfortunately is not async and the SerpAPI SDK does not support async either, so we much query the API directly using the `aiohttp` library.

In [6]:
import aiohttp

async with aiohttp.ClientSession() as session:
    async with session.get(
        "https://serpapi.com/search",
        params=params
    ) as response:
        results = await response.json()

results["organic_results"]

[{'position': 1,
  'title': 'World | Latest News & Updates',
  'link': 'https://www.bbc.com/news/world',
  'redirect_link': 'https://www.google.com/url?sa=t&source=web&rct=j&opi=89978449&url=https://www.bbc.com/news/world&ved=2ahUKEwjtroy37b6OAxWcrYkEHRQtLkgQFnoECDMQAQ',
  'displayed_link': 'https://www.bbc.com › news › world',
  'favicon': 'https://serpapi.com/searches/6876482e73bf59a3c53cdcf8/images/a1e5f2d48cb5e598291b88bdfff01798cff16e6d023b0c62d12765cd24da488e.png',
  'snippet': 'Israel bombs Syrian forces entering Druze city after sectarian clashes. About 100 people have reportedly been killed since fighting between Druze militias and ...',
  'snippet_highlighted_words': ['Israel bombs Syrian forces entering Druze city'],
  'sitelinks': {'inline': [{'title': 'BBC World', 'link': 'https://www.bbc.com/news/world_radio_and_tv'},
    {'title': 'Middle East', 'link': 'https://www.bbc.com/news/world/middle_east'},
    {'title': 'Europe', 'link': 'https://www.bbc.com/news/world/europe'}

With that, we have all we need to build a fully async serpapi tool.

In [8]:
from langchain_core.tools import tool

@tool
async def serpapi(query: str) -> list[Article]:
    """Use this tool to search the web."""
    params = {
        "api_key": SERPAPI_API_KEY,
        "engine": "google",
        "q": query,
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(
            "https://serpapi.com/search",
            params=params
        ) as response:
            results = await response.json() # 将json文本转换为python字典
    return [Article.from_serpapi_result(result) for result in results["organic_results"]]

Note that because this tool is async, we cannot use `tool.func` to call it as before:

In [9]:
serpapi.func

Instead, we use `tool.coroutine`:

In [10]:
serpapi.coroutine

<function __main__.serpapi(query: str) -> list[__main__.Article]>

---